In [7]:
import os
import zipfile
import tempfile
import re
from PIL import Image
import pandas as pd
import base64
from anthropic import Anthropic
import imghdr

# Set your Anthropic API key
client = Anthropic(api_key="YOUR_API_KEY")

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    image_type = imghdr.what(image_path)
    media_type = f"image/{image_type}"
    return encoded_string, media_type

def get_image_analysis(image_path):
    image_name = os.path.basename(image_path)
    
    with Image.open(image_path) as img:
        width, height = img.size
        channels = len(img.getbands())

    base64_image, media_type = encode_image(image_path)

    base_prompt = f"""Analyze the image titled '{image_name}' from a scientific paper. The image dimensions are {width}x{height} with {channels} channels.
Provide a comprehensive analysis structured as follows:
1. Image Type and Overview:
   - Identify the type of scientific figure (e.g., graph, diagram, microscopy image, flow chart).
   - Describe the general layout and main components of the image.
2. Key Scientific Concepts:
   - Identify and explain the main scientific concepts or principles illustrated in the image.
   - Discuss how these concepts relate to the likely field of study.
3. Data Representation:
   - If applicable, describe how data is represented (e.g., axes, scales, color coding).
   - Analyze any trends, patterns, or significant data points shown.
4. Methodology Illustration:
   - Identify any experimental setups, procedures, or methodologies depicted.
   - Explain how the image contributes to understanding the research methods used.
5. Results and Findings:
   - Interpret any results or findings presented in the image.
   - Discuss the potential implications of these results in the context of the research.
6. Technical Details:
   - Analyze any technical elements, such as equations, molecular structures, or specialized notations.
   - Explain their significance in the context of the research.
7. Visual Aids and Annotations:
   - Describe any labels, legends, or other annotations present in the image.
   - Explain how these elements enhance the understanding of the scientific content.
8. Interdisciplinary Connections:
   - Suggest potential connections to other scientific fields or disciplines.
   - Discuss how this research might impact or relate to broader scientific understanding.
9. Limitations and Considerations:
   - Identify any potential limitations or considerations in the data or methodology presented.
   - Suggest possible areas for further research or investigation based on the image.
10. Summary of Scientific Significance:
    - Provide a concise summary of the key scientific insights and importance of this image in research.
11. Summary:
    - Give a basic Summary of the Image
Please provide a detailed analysis for each section, ensuring a comprehensive understanding of the scientific content and significance of the image."""

    print(f"Sending image to Claude API for comprehensive analysis of {image_path}")
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=2000,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": media_type,
                                "data": base64_image
                            }
                        },
                        {
                            "type": "text",
                            "text": base_prompt
                        }
                    ]
                }
            ]
        )
        
        analysis = response.content[0].text.strip()
        print(f"Received comprehensive analysis from Claude API")
        return analysis
    except Exception as e:
        print(f"Error in get_image_analysis for {image_path}: {str(e)}")
        return f"Error: {str(e)}"

def get_figure_numbers(image_path):
    try:
        base64_image, media_type = encode_image(image_path)
        
        prompt = """### Instructions ###

You are an engineer skilled in Computer Vision and NLP. Your task is to identify the figure number or table number in the given image. 
Please respond using the specified format: lowercase "figure" or "table" followed by the lowercase number. Do not include a space between "table" and the number, or "figure" and the number.
Please provide your response strictly in the specified format, without including any additional text for formatting. I will use your response directly. If the number is unknown, provide an empty string "". Do not guess. 
If multiple figure or table numbers are found in the image, please find only the main figure or table number in the image(the one occupying the largest part of the image). Do not provide more than one response. 
The characters 1 and i can be easily confused in the image, but since only numbers should follow "table" or "figure," it should be interpreted as 1. There might be other confusing letters and numbers, but they should be interpreted as numbers.

### Example Output ###
"table1"

### Example Output ###
"table3"

### Example Output ###
"figure1"

### Example Output ###
"figure4"

### Example Output ###
""

### Output ###
"""
        
        print(f"Sending image to Claude API for figure/table number of {image_path}")
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=10,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": media_type,
                                "data": base64_image
                            }
                        },
                        {
                            "type": "text",
                            "text": prompt
                        }
                    ]
                }
            ]
        )
        
        result = response.content[0].text.strip().lower()
        print(f"Received response from Claude API: {result}")
        
        if result.startswith('figure'):
            figure_number = result[6:]
            table_number = ""
        elif result.startswith('table'):
            table_number = result[5:]
            figure_number = ""
        else:
            figure_number = ""
            table_number = ""
        
        return figure_number, table_number
    except Exception as e:
        print(f"Error in get_figure_numbers for {image_path}: {str(e)}")
        return "", ""

def extract_thesis_number(filename):
    match = re.match(r'(\d{3})_\d{3}_\d{3}', filename)
    if match:
        return match.group(1)
    return "Unknown"

def process_image(image_path):
    try:
        figure_number, table_number = get_figure_numbers(image_path)
        analysis = get_image_analysis(image_path)
        return figure_number, table_number, analysis
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return "", "", f"Error: {str(e)}"

def process_zip_file(zip_file_path):
    print(f"Processing zip file: {zip_file_path}")
    results = []
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        image_paths = []
        for root, dirs, files in os.walk(temp_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    image_paths.append(os.path.join(root, file))
        
        print(f"Found {len(image_paths)} images in zip file")
        if image_paths:
            for path in image_paths:
                thesis_number = extract_thesis_number(os.path.basename(path))
                figure_number, table_number, analysis = process_image(path)
                results.append({
                    'thesis_num': thesis_number,
                    'figure_num': figure_number,
                    'table_num': table_number,
                    'description': analysis
                })
        else:
            print("No image files found in the zip file.")
    
    return pd.DataFrame(results)

# Usage example
if __name__ == "__main__":
    try:
        zip_file_path = "snippets.zip"  # Make sure this matches your zip file name
        print(f"Starting processing of {zip_file_path}")
        df_results = process_zip_file(zip_file_path)
        
        if not df_results.empty:
            print("Results DataFrame:")
            print(df_results)
            # Optionally, save to CSV
            df_results.to_csv("image_analysis_results.csv", index=False)
            print("Results saved to image_analysis_results.csv")
        else:
            print("No results were returned.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

/var/folders/z_/c3rvbbwx2hsf19x07bkl8j800000gn/T/ipykernel_76625/3440838541.py:9: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


Starting processing of snippets.zip
Processing zip file: snippets.zip
Found 14 images in zip file
Sending image to Claude API for figure/table number of /var/folders/z_/c3rvbbwx2hsf19x07bkl8j800000gn/T/tmpx7tq8ipz/snippets/001_004_000.png
Received response from Claude API: ""
Sending image to Claude API for comprehensive analysis of /var/folders/z_/c3rvbbwx2hsf19x07bkl8j800000gn/T/tmpx7tq8ipz/snippets/001_004_000.png
Received comprehensive analysis from Claude API
Sending image to Claude API for figure/table number of /var/folders/z_/c3rvbbwx2hsf19x07bkl8j800000gn/T/tmpx7tq8ipz/snippets/001_014_000.png
Received response from Claude API: figure5
Sending image to Claude API for comprehensive analysis of /var/folders/z_/c3rvbbwx2hsf19x07bkl8j800000gn/T/tmpx7tq8ipz/snippets/001_014_000.png
Received comprehensive analysis from Claude API
Sending image to Claude API for figure/table number of /var/folders/z_/c3rvbbwx2hsf19x07bkl8j800000gn/T/tmpx7tq8ipz/snippets/001_005_000.png
Received respo